
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# GPU

 :label: `sec_use_gpu`

 :numref: `tab_intro_decade`では、過去 20 年間にわたる計算の急速な成長について説明しました。一言で言えば、GPU のパフォーマンスは 2000 年以来 10 年ごとに 1000 倍ずつ増加しています。これは大きなチャンスをもたらしますが、同時にそのようなパフォーマンスを提供することが非常に必要であることも示唆しています。

このセクションでは、この計算パフォーマンスを研究に活用する方法について説明します。最初は単一の GPU を使用し、その後で複数の GPU と複数のサーバー (複数の GPU を使用) を使用する方法について説明します。

具体的には、計算に単一の NVIDIA GPU を使用する方法について説明します。まず、少なくとも 1 つの NVIDIA GPU がインストールされていることを確認してください。次に、 [NVIDIA ドライバーと CUDA](https://developer.nvidia.com/cuda-downloads)をダウンロードし、プロンプトに従って適切なパスを設定します。これらの準備が完了すると、 `nvidia-smi`コマンドを使用して (**グラフィックス カード情報を表示**) できるようになります。


In [1]:
!nvidia-smi

Fri Feb 10 06:11:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.106.00   Driver Version: 460.106.00   CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:17.0 Off |                    0 |
| N/A   35C    P0    76W / 300W |   1534MiB / 16160MiB |     53%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+


|   1  Tesla V100-SXM2...  Off  | 00000000:00:18.0 Off |                    0 |
| N/A   34C    P0    42W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   2  Tesla V100-SXM2...  Off  | 00000000:00:19.0 Off |                    0 |
| N/A   36C    P0    80W / 300W |   3308MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+


|   3  Tesla V100-SXM2...  Off  | 00000000:00:1A.0 Off |                    0 |
| N/A   35C    P0   200W / 300W |   3396MiB / 16160MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   4  Tesla V100-SXM2...  Off  | 00000000:00:1B.0 Off |                    0 |
| N/A   32C    P0    56W / 300W |   1126MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+


|   5  Tesla V100-SXM2...  Off  | 00000000:00:1C.0 Off |                    0 |
| N/A   40C    P0    84W / 300W |   1522MiB / 16160MiB |     47%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   6  Tesla V100-SXM2...  Off  | 00000000:00:1D.0 Off |                    0 |
| N/A   34C    P0    57W / 300W |    768MiB / 16160MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   7  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P0    41W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                        

+-----------------------------------------------------------------------------+



PyTorch では、すべての配列にデバイスがあり、それをコンテキストと呼ぶことがよくあります。これまでのところ、デフォルトでは、すべての変数と関連する計算が CPU に割り当てられています。通常、他のコンテキストはさまざまな GPU である可能性があります。複数のサーバーにジョブをデプロイする場合、事態はさらに困難になる可能性があります。配列をコンテキストにインテリジェントに割り当てることで、デバイス間のデータ転送にかかる時間を最小限に抑えることができます。たとえば、GPU を備えたサーバー上でニューラル ネットワークをトレーニングする場合、通常はモデルのパラメーターが GPU 上に存在することを好みます。



このセクションのプログラムを実行するには、少なくとも 2 つの GPU が必要です。これはほとんどのデスクトップ コンピューターにとって贅沢かもしれませんが、AWS EC2 マルチ GPU インスタンスを使用するなど、クラウドで簡単に利用できることに注意してください。他のほとんどすべてのセクションでは複数の GPU は必要あり*ません*。代わりに、これは単に、異なるデバイス間でデータがどのように流れるかを示すためのものです。


In [2]:
import torch
from torch import nn
from d2l import torch as d2l


## [**コンピューティングデバイス**]

ストレージや計算のために CPU や GPU などのデバイスを指定できます。デフォルトでは、テンソルはメイン メモリに作成され、CPU を使用して計算されます。



PyTorch では、CPU と GPU `torch.device(&#39;cpu&#39;)`と`torch.device(&#39;cuda&#39;)`で指定できます。なお、 `cpu`デバイスとは物理的な CPU やメモリをすべて指します。これは、PyTorch の計算がすべての CPU コアを使用しようとすることを意味します。ただし、 `gpu`デバイスは 1 つのカードと対応するメモリのみを表します。複数の GPU がある場合、 `torch.device(f&#39;cuda:{i}&#39;)`を使用して $i^\mathrm{th}$ GPU を表します ($i$ は 0 から始まります)。また、 `gpu:0`と`gpu`同等です。


In [3]:
def cpu():  #@save
    """Get the CPU device."""
    return torch.device('cpu')

def gpu(i=0):  #@save
    """Get a GPU device."""
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))


(**利用可能な GPU の数をクエリします。** )


In [4]:
def num_gpus():  #@save
    """Get the number of available GPUs."""
    return torch.cuda.device_count()

num_gpus()

2


ここで、[**要求された GPU が存在しない場合でもコードを実行できるようにする 2 つの便利な関数を定義します。** 】


In [5]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0), device(type='cuda', index=1)])


## テンソルとGPU



デフォルトでは、テンソルは CPU 上に作成されます。 [**テンソルが配置されているデバイスをクエリできます。** 】


In [6]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')


複数の用語を操作したい場合は常に、それらの用語が同じデバイス上にある必要があることに注意することが重要です。たとえば、2 つのテンソルを合計する場合、両方の引数が同じデバイス上に存在することを確認する必要があります。そうしないと、フレームワークは結果をどこに保存するか、あるいは計算を実行する場所を決定する方法さえも認識できなくなります。

###  GPU上のストレージ

[ **GPU にテンソルを保存するには、いくつかの方法があります。** ] たとえば、テンソルを作成するときにストレージ デバイスを指定できます。次に、最初の`gpu`上にテンソル変数`X`を作成します。 GPU で作成されたテンソルは、この GPU のメモリのみを消費します。 `nvidia-smi`コマンドを使用して、GPU メモリの使用状況を表示できます。一般に、GPU メモリ制限を超えるデータを作成しないようにする必要があります。


In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


少なくとも 2 つの GPU があると仮定すると、次のコードは ( **2 番目の GPU にランダムなテンソルを作成します。** )


In [8]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.7781, 0.1400, 0.4503],
        [0.1745, 0.2343, 0.6356]], device='cuda:1')


### コピーする

[ **`X + Y`を計算したい場合は、この操作をどこで実行するかを決定する必要があります。** ] たとえば、 :numref: `fig_copyto`に示すように、 `X` 2 番目の GPU に転送し、そこで操作を実行できます。例外が発生するため、単純に`X`と`Y`を追加し*ないでください*。ランタイム エンジンは何をすべきかわかりません。同じデバイス上でデータが見つからず、失敗します。 `Y` 2 番目の GPU 上に存在するため、2 つを追加する前に`X`そこに移動する必要があります。 

![](http://d2l.ai/_images/copyto.svg) :label: `fig_copyto`


In [9]:
Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')



[**データが同じ GPU ( `Z`と`Y`の両方) 上にあるので、それらを合計できます。** 】


In [10]:
Y + Z

tensor([[1.7781, 1.1400, 1.4503],
        [1.1745, 1.2343, 1.6356]], device='cuda:1')


変数`Z`がすでに 2 番目の GPU に存在していると想像してください。それでも`Z.cuda(1)`を呼び出すとどうなるでしょうか?コピーを作成して新しいメモリを割り当てる代わりに、 `Z`を返します。


In [11]:
Z.cuda(1) is Z

True


### サイドノート

人々は、GPU が高速であることを期待して、機械学習を行うために GPU を使用します。ただし、デバイス間での変数の転送には時間がかかります。したがって、私たちは、何かをゆっくりと実行したいと考えていることを 100% 確信してから実行してもらいたいと考えています。深層学習フレームワークがクラ​​ッシュせずにコピーを自動的に実行しただけであれば、遅いコードを書いたことに気づかない可能性があります。

また、デバイス (CPU、GPU、その他のマシン) 間のデータ転送は、計算よりもはるかに遅くなります。また、さらに操作を進める前にデータが送信される (または受信される) まで待たなければならないため、並列化がさらに困難になります。このため、コピー操作は細心の注意を払って行う必要があります。経験則として、多くの小さな操作は 1 つの大きな操作よりもはるかに悪いです。さらに、何をやっているのかよくわかっていない限り、一度に複数の操作を実行するほうが、コード内に多数の単一操作を散在させるよりもはるかに優れています。これは、一方のデバイスが他のことを行う前に他方のデバイスを待機する必要がある場合、そのような操作がブロックされる可能性があるためです。これは、電話でコーヒーを事前注文して、準備ができていることがわかるのではなく、行列に並んでコーヒーを注文するのと似ています。

最後に、テンソルを出力するとき、またはテンソルを NumPy 形式に変換するときに、データがメイン メモリにない場合、フレームワークは最初にデータをメイン メモリにコピーするため、追加の送信オーバーヘッドが発生します。さらに悪いことに、Python が完了するまですべてを待たせる恐ろしいグローバル インタプリタ ロックの対象になっています。

##  [**ニューラルネットワークとGPU** ]

同様に、ニューラル ネットワーク モデルはデバイスを指定できます。次のコードは、モデル パラメーターを GPU に配置します。


In [12]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())


次の章では、GPU でモデルを実行する方法の例をさらに多く見ていきます。これは、単に計算量が多少多くなるからです。

入力が GPU 上のテンソルである場合、モデルは同じ GPU 上で結果を計算します。


In [13]:
net(X)

tensor([[0.1746],
        [0.1746]], device='cuda:0', grad_fn=<AddmmBackward0>)


(**モデルパラメータが同じ GPU に保存されていることを確認してみましょう。** )


In [14]:
net[0].weight.data.device

device(type='cuda', index=0)


トレーナーに GPU をサポートさせます。


In [15]:
@d2l.add_to_class(d2l.Trainer)  #@save
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model


つまり、すべてのデータとパラメーターが同じデバイス上にある限り、モデルを効率的に学習できます。次の章では、そのような例をいくつか見ていきます。

## まとめ

CPU や GPU など、ストレージと計算用のデバイスを指定できます。デフォルトでは、データはメイン メモリに作成され、CPU が計算に使用されます。ディープ ラーニング フレームワークでは、計算用のすべての入力データが同じデバイス (CPU または同じ GPU) 上に存在する必要があります。データを不注意に移動すると、パフォーマンスが大幅に低下する可能性があります。典型的な間違いは次のとおりです。GPU 上のすべてのミニバッチの損失を計算し、それをコマンド ラインでユーザーに報告する (または NumPy `ndarray`に記録する) と、グローバル インタプリタ ロックがトリガーされ、すべての GPU が停止します。 GPU 内のログ用にメモリを割り当て、より大きなログのみを移動する方がはるかに優れています。

## 演習
1. 大きな行列の乗算など、より大きな計算タスクを試して、CPU と GPU の速度の違いを確認してください。計算量が少ないタスクの場合はどうでしょうか?
1.  GPU 上でモデル パラメーターを読み書きするにはどうすればよいでしょうか?
1.  $100 \times 100$ 行列の 1,000 回の行列間の乗算を計算し、出力行列のフロベニウス ノルムを一度に 1 つの結果ずつログに記録するのにかかる時間を、GPU にログを保持して最終結果のみを転送する場合と比べて測定します。
1.  2 つの行列間の乗算を 2 つの GPU で同時に実行する場合と、1 つの GPU で順番に実行する場合にかかる時間を測定します。ヒント: ほぼ線形のスケーリングが表示されるはずです。



[ディスカッション](https://discuss.d2l.ai/t/63)
